In [8]:
# Cellar.ai Wine Recommendation Prototype
# Save this as: wine_recommendation_prototype.ipynb
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

2025-07-15 22:43:38.158876: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-15 22:43:38.168701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752583418.179673 1126486 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752583418.182885 1126486 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752583418.191861 1126486 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [10]:
# ===============================
# 0. CONSTS
# ===============================

vintage_min = 1985
vintage_max = 2025

In [ ]:
# ===============================
# 2. LOAD VIVINO  
# ===============================

    




In [1]:
df_vv = VivinoStoreLoader(path_pattern='data/review/vivino**.csv').load()

df_vm = WinemagStoreLoader(path_pattern='data/review/winemag**.csv').load()
# vm_df = VivinoWineLoader(path_pattern='../data/winemag**.csv').load()


df = pd.concat([df_vv, df_vm], axis=0)

print(df.dtypes)

df.to_csv('data/store/master.csv')

# df = df.dropna(subset=['variety'])


# df = df[(df['year'] >= vintage_min) & (df['year'] <= vintage_max)]

# # Basic exploration
# print(f"\n📊 Dataset Overview:")
# print(f"   Total wines: {len(df):,}")
# print(f"   Columns: {list(df.columns)}")
# print(f"   Price range: ${df['price'].min():.0f} - ${df['price'].max():.0f}")
# # print(f"   Regions: {list(df['province'].drop_duplicates())}")
# print(f"   Varietal: {list(df['variety'].drop_duplicates())}")
# print(f"   Year range: {df['year'].min():.0f} - {df['year'].max():.0f}")


# # Create a scatter plot of the data. To change the markers to red "x",
# # we used the 'marker' and 'c' parameters
# df = df[df['price'] <= 100]
# df = df[df['type'] == 'red'].copy()
# top_20 = df['variety'].value_counts().head(20)
# df = df[df['variety'].isin(top_20.index.tolist())]

# df = df[df['rating'] >= 0]

# df = df.sort_values(by=['rating'], ascending=False)

# display(df.head(20))



NameError: name 'VivinoStoreLoader' is not defined

In [14]:

# PLOT DATA
# plt.scatter(df['price'], df['rank'], marker='x', c='r') 

# # Set the title
# plt.title("rank vs. Price")
# # Set the y-axis label
# plt.ylabel('Y')
# # Set the x-axis label
# plt.xlabel('X')
# plt.show()

# Show sample wines
print(f"\n🍾 Sample Wines:")
# display(df.head())

# LINEAR REG PARAMS

# features = ['province', 'vintage', 'variety'] # ['province', 'vintage', 'country', 'variety'] 

# do the reg only for most popular reds
df = df[df['type'] == 'red'].copy()
top_20 = df['variety'].value_counts().head(20)
df = df[df['variety'].isin(top_20.index.tolist())]

target = 'price'
features = ['year', 'variety']

# remove outlier for y (price)
df = df[df[target] <= 100]

# Drop first helps with multicollinearity, i.e. when 2 features are correlated and their impact on y cannot be detangled
df = pd.get_dummies(df, columns=['variety'], drop_first=True)

# df = pd.get_dummies(df, columns=['province'], drop_first=True)

# For the year, we create 2x features, one for the age (normalized 0..1) and one for the vintage (one-hot)
scaler = StandardScaler()
df['age'] = vintage_max - df['year']
df['age_normalized'] = scaler.fit_transform(df[['age']])

df = pd.get_dummies(df, columns=['year'], drop_first=True)



# points


encoded_features = [col for col in df.columns 
                   if any(col.startswith(prefix + '_') for prefix in features)]


X = df[encoded_features + ['age_normalized', 'rank']] # 
y = df[target]


print(X.shape)


print(f"   Total wines: {len(X):,}")
print(f"   Features: {list(X.columns)}")
print(f"   Price range: ${y.min():.0f} - ${y.max():.0f}")
print(f"   Age range: {df['age'].min():.0f} - {df['age'].max():.0f}")


# 2. Split the data
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # 3. Create and train the model
# from sklearn.linear_model import LinearRegression
# model = LinearRegression()
# model.fit(X_train, y_train)

# # 4. Make predictions
# y_pred = model.predict(X_test)

# # 5. Evaluate
# from sklearn.metrics import r2_score, mean_squared_error
# r2 = r2_score(y_test, y_pred)
# rmse = np.sqrt(mean_squared_error(y_test, y_pred))


# What this means for your results:
# RMSE = 25.056:

# Direct measure of cost function performance
# "On average, predictions are off by $25"

# R² = 0.038:

# Your model's error vs baseline error
# "Your model is only 3.8% better than just predicting the mean price ($40.46) every time"
# print(f"R² Score: {r2:.3f}")
# print(f"RMSE: {rmse:.3f}")

# print(f"Target range: {y.min()} to {y.max()}")
# print(f"Target mean: {y.mean():.2f}")
# print(f"Number of features: {X.shape[1]}")
# print(f"Training samples: {X.shape[0]}")


# Search for wine/winery
# df = df[df['title'].str.contains('sister', case=False, na=False)]
# df = df[df['title'].str.contains('run', case=False, na=False)]
# df = df[df['province'].str.contains('Australia Other', case=False, na=False)]

# display(df.head(30))




🍾 Sample Wines:


NameError: name 'df' is not defined

In [ ]:

# Create correlation matrix for your features
# correlation_matrix = X.corr()
# mask = np.abs(correlation_matrix) < 0.5  # Hide weak correlations

# # Plot the heatmap
# plt.figure(figsize=(12, 10))
# sns.heatmap(correlation_matrix, mask=mask, annot=True, cmap='coolwarm')
# plt.title('Strong Correlations Only (>0.5)')
# plt.show()

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.regularizers import L2 

# convert bools to 0/1
cs = X.select_dtypes(include=['bool']).columns
X[cs] = X[cs].astype(int)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Price range: ${y.min():.0f} to ${y.max():.0f}")
print(f"Average price: ${y.mean():.0f}")
print(f"Price std dev: ${y.std():.0f}")


Y_train = np.array(y_train)
X_train = X_train.to_numpy()

Y_test = np.array(y_test)
X_test = X_test.to_numpy()
display(X_test)




# Build the model, 88 features
# model = Sequential([
#     Dense(100, activation='relu', input_shape=(38,)),  # Hidden layer
#     Dense(50, activation='relu'),  # Hidden layer
#     Dense(20, activation='relu'),  # Hidden layer
#     Dense(1, activation='linear')                     # Output layer
# ])

model = Sequential([
    Dense(500, activation='relu', input_shape=(38,)),
    Dense(200, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(50, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(20, activation='relu', kernel_regularizer=L2(0.01)),
    Dense(1, activation='linear', kernel_regularizer=L2(0.01))
])


#print(X_train)
# print(Y_train)

model.compile(loss=MeanSquaredError, metrics=['mae', 'mse'])
model.fit(X_train, Y_train, epochs=50, validation_split=0.2)

test_loss, test_mae, test_mse = model.evaluate(X_test, Y_test, verbose=0)
print(f"Test Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")
print(f"Test MSE: {test_mse:.4f}")



Price range: $2 to $100
Average price: $25
Price std dev: $21


array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.81117488,  0.47826087],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.47111904,  0.47826087],
       [ 0.        ,  0.        ,  1.        , ...,  0.        ,
         1.2291602 ,  0.56521739],
       ...,
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
        -0.47111904,  0.47826087],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.5490485 ,  0.82608696],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
        -0.13106319,  0.56521739]])

Epoch 1/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 483.7198 - mae: 14.9252 - mse: 480.0356 - val_loss: 240.4683 - val_mae: 10.0613 - val_mse: 237.3746
Epoch 2/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 247.3144 - mae: 10.7475 - mse: 244.2004 - val_loss: 196.0710 - val_mae: 9.1208 - val_mse: 192.8678
Epoch 3/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 191.1298 - mae: 9.4271 - mse: 187.8823 - val_loss: 200.8545 - val_mae: 8.5899 - val_mse: 197.4660
Epoch 4/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 188.4757 - mae: 8.9806 - mse: 185.0564 - val_loss: 181.7294 - val_mae: 8.6198 - val_mse: 178.2408
Epoch 5/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 175.8345 - mae: 8.6749 - mse: 172.3331 - val_loss: 179.5156 - val_mae: 8.5856 - val_mse: 175.9738
Epoch 6/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 174.0688 - mae: 8.6297 - mse: 170.5190 - val_loss: 178.8027 - val_mae: 8.5108 - val_mse: 175.2512
Epoch 7/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/st

In [ ]:

# print(Y_test.shape)
# print(X_test.shape)

# for i in range(Y_test.shape[0]):
#     print(X_test[i,:].shape)
#     p = model.predict(X_test[i,:])
#     print(p)